# Sachsen-Anhalt

Every federal state is represented by its own input directory and is processed into a NUTS level 2 directory containing a sub-folder for each discharge location. These folder names are derived from NUTS and reflect the CAMELS id. The NUTS level 2 code for Sachsen-Anhalt is `DEE`.

To pre-process the data, you need to write (at least) two functions. One should extract all metadata and condense it into a single `pandas.DataFrame`. This is used to build the folder structure and derive the ids.
The second function has to take an id, as provided by the state authorities, called `provider_id` and return a `pandas.DataFrame` with the transformed data. The dataframe needs the three columns `['date', 'q' | 'w', 'flag']`.

For easier and unified output handling, the `camelsp` package contains a context object called `Bundesland`. It takes a number of names and abbreviations to identify the correct federal state and returns an object that holds helper and save functions.

The context saves files as needed and can easily be changed to save files with different strategies, ie. fill missing data with NaN, merge data into a single file, create files for each variable or pack everything together into a netcdf.

In [1]:
import pandas as pd
import geopandas as gpd
import os
import shutil
from glob import glob
from tqdm import tqdm
import patoolib
from typing import Dict, Tuple
import warnings

from camelsp import Bundesland

The context can also be instantiated as any regular Python class, ie. to load only the default input data path, that we will user later.

In [2]:
# the context also makes the input path available, if camelsp was install locally
BASE = Bundesland('Sachsen-Anhalt').input_path
BASE

'/home/camel/camelsp/input_data/Q_and_W/SA_Sachsen-Anhalt'

## Parse data

We do not have a Metadata file, but one Excel file for each station. Thus we need to parse each metadata individually and collect.

In [3]:
# remove folder TagMittel_DGJ_20221007 to ensure that data is extracted freshly and is up to date
if os.path.exists(f"{BASE}/TagMittel_DGJ_20221007"):
    shutil.rmtree(f"{BASE}/TagMittel_DGJ_20221007")

# extract .7z archive, make sure that 7z is installed to make patoolib work
patoolib.extract_archive(f"{BASE}/TagMittel_DGJ_LSA.7z", outdir=BASE)

patool: Extracting /home/camel/camelsp/input_data/Q_and_W/SA_Sachsen-Anhalt/TagMittel_DGJ_LSA.7z ...
patool: running /usr/bin/7z x -o/home/camel/camelsp/input_data/Q_and_W/SA_Sachsen-Anhalt -- /home/camel/camelsp/input_data/Q_and_W/SA_Sachsen-Anhalt/TagMittel_DGJ_LSA.7z
patool: ... /home/camel/camelsp/input_data/Q_and_W/SA_Sachsen-Anhalt/TagMittel_DGJ_LSA.7z extracted to `/home/camel/camelsp/input_data/Q_and_W/SA_Sachsen-Anhalt'.


'/home/camel/camelsp/input_data/Q_and_W/SA_Sachsen-Anhalt'

In [4]:
files = sorted(glob(os.path.join(f"{BASE}/TagMittel_DGJ_20221007", 'LHW_*.DGJ')))
print(f"Found {len(files)} files.")

Found 252 files.


Check the Header on the first file, as we don't have any other Metadata file. We need a ZRXP parser. Couldn't find something simple quickly, so I write my own. https://prozessing.tbbm.at/zrxp/zrxp3.0_en.pdf this is the ZRXP specification. I will only implement the relevant metadata

In [5]:
# get the names from the header
# I will skip the energy market headers and remote logger headers
HEADER = dict(
    SANR='Alphanumerical station number',
    SNAME='Station name',
    SWATER='River name',
    CMW='Values per day for equidistant time series values',
    CNAME='Parameter name',
    CNR='Parameter number',
    CUNIT='Unit of the data value column',
    RINVAL='Value for missing or invalid data record',
    RTIMELVL='Time series time level',
    TZ='time zone of all time stamps in the time series block, both header and data',
    LAYOUT='specifies the column layout for the ZRXP data'
)

def extract_file(path: str) -> Tuple[Dict[str, str], pd.DataFrame]:
    # Get the header lines first
    collection = []
    headerlines = 0

    # first read the head
    with open(path, 'rb') as f:
        # go for each line
        for l in f.readlines():
            if not l.decode('latin1').startswith('#'):
                break
            else:
                # collect the header
                collection.extend([_ for _ in l.decode('latin1').replace('#', '').split('|*|') if _ not in ('', '\n', '\r\n')])
                headerlines += 1

    # create metadata container
    meta = {}

    # go for each collected header
    for co in collection:
        HEAD = [k for k in HEADER.keys() if co.startswith(k)]
        if len(HEAD) == 1:
            HEAD = HEAD[0]
            meta[HEAD] = co.replace(HEAD, '')
        elif len(HEAD) == 0:
            if 'COMMENT' in meta:
                meta['COMMENT'] += f" {co}"
            else:
                meta["COMMENT"] = co
        elif len(HEAD) > 1:
            warnings.warn(f"Can't parse header {co}")

    # now the data
    header = meta['LAYOUT'].strip('()').split(',') if 'LAYOUT' in meta else [0, 1, 2, 3, 4, 5]
    df = pd.read_csv(path, encoding='latin1', sep=' ', header=None, skiprows=headerlines, names=header, parse_dates=[0], na_values=int(meta.get('RINVAL', '-777')))

    return meta, df

extract_file(files[-1])


({'COMMENT': 'ZRXPVERSION2300.100 ZRXPCREATORKiIOSystem.ZRXPV2R2_E SOURCESYSTEMWISKI SOURCEIDf567eda8-1628-42ec-89b9-fac8a7c8b760 TSPATH/LHW/597305/W.DGJ/Day.Mean.DGJ',
  'SANR': '597305',
  'SNAME': 'Ritzleben',
  'SWATER': 'Flötgraben',
  'CNR': 'W.DGJ',
  'CNAME': 'W.DGJ',
  'TZ': 'UTC+1',
  'RINVAL': '-777',
  'CUNIT': 'cm',
  'LAYOUT': '(timestamp,value,status,interpolation_type,remark)'},
        timestamp  value  status  interpolation_type remark
 0     1984-11-01     30      40                 603    NaN
 1     1984-11-02     30      40                 603    NaN
 2     1984-11-03     30      40                 603    NaN
 3     1984-11-04     30      40                 603    NaN
 4     1984-11-05     29      40                 603    NaN
 ...          ...    ...     ...                 ...    ...
 13570 2021-12-27     36      40                 603    NaN
 13571 2021-12-28     35      40                 603    NaN
 13572 2021-12-29     34      40                 603    NaN
 1

Go go for all files

In [6]:
# container
meta = []
raw_data = []

with warnings.catch_warnings(record=True) as warn:
    for fname in tqdm(files):
        m, df = extract_file(fname)
        meta.append(m)
        raw_data.append(df)

print(f"Parsed {len(meta)} files with {len(warn)} warnings.")

100%|██████████| 252/252 [00:18<00:00, 13.91it/s]

Parsed 252 files with 0 warnings.


# create metadata

This should be pretty straightforward, but maybe not super-helpful

In [7]:
metadata = pd.DataFrame(meta)
metadata

,COMMENT,SANR,SNAME,SWATER,CNR,CNAME,TZ,RINVAL,CUNIT,LAYOUT
0,ZRXPVERSION2300.100 ZRXPCREATORKiIOSystem.ZRXP...,440003,Ummendorf,Aller,Q.DGJ,Q.DGJ,UTC+1,-777,m³/s,"(timestamp,value,status,interpolation_type,rem..."
1,ZRXPVERSION2300.100 ZRXPCREATORKiIOSystem.ZRXP...,440003,Ummendorf,Aller,W.DGJ,W.DGJ,UTC+1,-777,cm,"(timestamp,value,status,interpolation_type,rem..."
2,ZRXPVERSION2300.100 ZRXPCREATORKiIOSystem.ZRXP...,440004,Alleringersleben,Aller,Q.DGJ,Q.DGJ,UTC+1,-777,m³/s,"(timestamp,value,status,interpolation_type,rem..."
3,ZRXPVERSION2300.100 ZRXPCREATORKiIOSystem.ZRXP...,440004,Alleringersleben,Aller,W.DGJ,W.DGJ,UTC+1,-777,cm,"(timestamp,value,status,interpolation_type,rem..."
4,ZRXPVERSION2300.100 ZRXPCREATORKiIOSystem.ZRXP...,440008,Walbeck,Aller,Q.DGJ,Q.DGJ,UTC+1,-777,m³/s,"(timestamp,value,status,interpolation_type,rem..."
...,...,...,...,...,...,...,...,...,...,...
247,ZRXPVERSION2300.100 ZRXPCREATORKiIOSystem.ZRXP...,597205,Tylsen,Salzwedeler Dumme,W.DGJ,W.DGJ,UTC+1,-777,cm,"(timestamp,value,status,interpolation_type,rem..."
248,ZRXPVERSION2300.100 ZRXPCREATORKiIOSystem.ZRXP...,597208,Salzwedel-Dumme,Salzwedeler Dumme,Q.DGJ,Q.DGJ,UTC+1,-777,m³/s,"(timestamp,value,status,interpolation_type,rem..."
249,ZRXPVERSION2300.100 ZRXPCREATORKiIOSystem.ZRXP...,597208,Salzwedel-Dumme,Salzwedeler Dumme,W.DGJ,W.DGJ,UTC+1,-777,cm,"(timestamp,value,status,interpolation_type,rem..."
250,ZRXPVERSION2300.100 ZRXPCREATORKiIOSystem.ZRXP...,597305,Ritzleben,Flötgraben,Q.DGJ,Q.DGJ,UTC+1,-777,m³/s,"(timestamp,value,status,interpolation_type,rem..."


There is more metadata in the provided Pegel shapefile, including critical information such as location.  
In the case of Sachsen-Anhalt, the provided geopackage has two layers; one for the catchments, and one for the stations, we add both to the general metadata.

In [8]:
# read geopackage, layer Pegel_EZG
gdf_meta_ezg = gpd.read_file(os.path.join(BASE, '../../Shapes/Sachsen-Anhalt_Shapes/Datensatz.gpkg'), layer='Pegel_EZG')

# drop column geometry
gdf_meta_ezg.drop(columns='geometry', inplace=True)

# provider_id as int
gdf_meta_ezg['PKZ'] = gdf_meta_ezg['PKZ'].astype(int)
metadata['SANR'] = metadata['SANR'].astype(int)

# concat metadata and gdf_meta_ezg
metadata = metadata.merge(gdf_meta_ezg, left_on='SANR', right_on='PKZ', how='left')

# make Pegelkennziffer str again
metadata['SANR'] = metadata['SANR'].astype(str)

In [9]:
# read geopackage, layer Pegel_LHW
gdf_meta_pegel = gpd.read_file(os.path.join(BASE, '../../Shapes/Sachsen-Anhalt_Shapes/Datensatz.gpkg'), layer='Pegel_LHW')

# drop column geometry
gdf_meta_pegel.drop(columns='geometry', inplace=True)

# drop columns that we already have in metadata
gdf_meta_pegel.drop(columns=['ZR', 'Name', 'Gewässer'], inplace=True)

# provider_id as int
gdf_meta_pegel['PKZ'] = gdf_meta_pegel['PKZ'].astype(int)
metadata['SANR'] = metadata['SANR'].astype(int)

# concat metadata and gdf_meta_pegel
metadata = metadata.merge(gdf_meta_pegel, left_on='SANR', right_on='PKZ', how='left')

# make Pegelkennziffer str again
metadata['SANR'] = metadata['SANR'].astype(str)

In [10]:
id_column = 'SANR'

Metadata is seperated for Q and W data, this would result in two seperate entries in the nuts_mapping for each station:

In [11]:
print(f"Two entries for ID '578410':\n{metadata[metadata[id_column] == '578410']}")

# all IDs have two entries -> Q and W
print(f"\nNumber of duplicated IDs: {metadata[id_column].duplicated().sum()}")


Two entries for ID '578410':
                                              COMMENT    SANR   SNAME  SWATER  \
86  ZRXPVERSION2300.100 ZRXPCREATORKiIOSystem.ZRXP...  578410  Wippra  Wipper   
87  ZRXPVERSION2300.100 ZRXPCREATORKiIOSystem.ZRXP...  578410  Wippra  Wipper   

      CNR  CNAME     TZ RINVAL CUNIT  \
86  Q.DGJ  Q.DGJ  UTC+1   -777  m³/s   
87  W.DGJ  W.DGJ  UTC+1   -777    cm   

                                               LAYOUT  ...  WEG_3ST WEG_4ST  \
86  (timestamp,value,status,interpolation_type,rem...  ...      567    5676   
87  (timestamp,value,status,interpolation_type,rem...  ...      567    5676   

    EZG außerhalb LSA Betreiber   PKZ_y     Auflösung Northing  Easting  \
86               nein       LHW  578410  Day.Mean.DGJ  5716322   657899   
87               nein       LHW  578410  Day.Mean.DGJ  5716322   657899   

   Beeinflussung durch  PNP mNN  
86    Vorsperre Wippra   242.62  
87    Vorsperre Wippra   242.62  

[2 rows x 23 columns]

Number of duplic

We have to merge the Q and W metadata first.

In [12]:
# the following columns are Q/W specific and get a Q_ or W_ prefix
update_keys = ['COMMENT', 'CNR', 'CNAME', 'CUNIT']

# get all unique ids
all_ids = metadata[id_column].unique()

# container
qw_metadata = []

# go for each id
for id in all_ids:
    # get Q metadata as dict
    q_meta = metadata[(metadata[id_column] == id) & (metadata['CNAME'] == 'Q.DGJ')].to_dict(orient='records')[0]
    # get W metadata as dict
    w_meta = metadata[(metadata[id_column] == id) & (metadata['CNAME'] == 'W.DGJ')].to_dict(orient='records')[0]

    q_updated_meta = {}
    w_updated_meta = {}

    # add prefix to Q keys
    for key in q_meta.keys():
        if key in update_keys:
            q_updated_meta[f"Q_{key}"] = q_meta[key]
        else:
            q_updated_meta[key] = q_meta[key]
    
    # add prefix to W keys
    for key in w_meta.keys():
        if key in update_keys:
            w_updated_meta[f"W_{key}"] = w_meta[key]
        else:
            w_updated_meta[key] = w_meta[key]

    # merge Q and W metadata
    
    qw_metadata.append({**q_updated_meta, **w_updated_meta})

qw_metadata = pd.DataFrame(qw_metadata)
qw_metadata
    

,Q_COMMENT,SANR,SNAME,SWATER,Q_CNR,Q_CNAME,TZ,RINVAL,Q_CUNIT,LAYOUT,...,PKZ_y,Auflösung,Northing,Easting,Beeinflussung durch,PNP mNN,W_COMMENT,W_CNR,W_CNAME,W_CUNIT
0,ZRXPVERSION2300.100 ZRXPCREATORKiIOSystem.ZRXP...,440003,Ummendorf,Aller,Q.DGJ,Q.DGJ,UTC+1,-777,m³/s,"(timestamp,value,status,interpolation_type,rem...",...,440003,Day.Mean.DGJ,5780490,649741,NaN,124.90,ZRXPVERSION2300.100 ZRXPCREATORKiIOSystem.ZRXP...,W.DGJ,W.DGJ,cm
1,ZRXPVERSION2300.100 ZRXPCREATORKiIOSystem.ZRXP...,440004,Alleringersleben,Aller,Q.DGJ,Q.DGJ,UTC+1,-777,m³/s,"(timestamp,value,status,interpolation_type,rem...",...,440004,Day.Mean.DGJ,5786828,645809,NaN,113.24,ZRXPVERSION2300.100 ZRXPCREATORKiIOSystem.ZRXP...,W.DGJ,W.DGJ,cm
2,ZRXPVERSION2300.100 ZRXPCREATORKiIOSystem.ZRXP...,440008,Walbeck,Aller,Q.DGJ,Q.DGJ,UTC+1,-777,m³/s,"(timestamp,value,status,interpolation_type,rem...",...,440008,Day.Mean.DGJ,5793982,641118,NaN,94.34,ZRXPVERSION2300.100 ZRXPCREATORKiIOSystem.ZRXP...,W.DGJ,W.DGJ,cm
3,ZRXPVERSION2300.100 ZRXPCREATORKiIOSystem.ZRXP...,440010,Weferlingen,Aller,Q.DGJ,Q.DGJ,UTC+1,-777,m³/s,"(timestamp,value,status,interpolation_type,rem...",...,440010,Day.Mean.DGJ,5797996,640170,NaN,84.32,ZRXPVERSION2300.100 ZRXPCREATORKiIOSystem.ZRXP...,W.DGJ,W.DGJ,cm
4,ZRXPVERSION2300.100 ZRXPCREATORKiIOSystem.ZRXP...,441201,Hödingen,Schölecke,Q.DGJ,Q.DGJ,UTC+1,-777,m³/s,"(timestamp,value,status,interpolation_type,rem...",...,441201,Day.Mean.DGJ,5796366,643497,NaN,93.97,ZRXPVERSION2300.100 ZRXPCREATORKiIOSystem.ZRXP...,W.DGJ,W.DGJ,cm
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121,ZRXPVERSION2300.100 ZRXPCREATORKiIOSystem.ZRXP...,597008,Sienau UP,Jeetze,Q.DGJ,Q.DGJ,UTC+1,-777,m³/s,"(timestamp,value,status,interpolation_type,rem...",...,597008,Day.Mean.DGJ,5855172,646513,Wehr,18.39,ZRXPVERSION2300.100 ZRXPCREATORKiIOSystem.ZRXP...,W.DGJ,W.DGJ,cm
122,ZRXPVERSION2300.100 ZRXPCREATORKiIOSystem.ZRXP...,597105,Hagen,Purnitz,Q.DGJ,Q.DGJ,UTC+1,-777,m³/s,"(timestamp,value,status,interpolation_type,rem...",...,597105,Day.Mean.DGJ,5847185,646003,NaN,23.61,ZRXPVERSION2300.100 ZRXPCREATORKiIOSystem.ZRXP...,W.DGJ,W.DGJ,cm
123,ZRXPVERSION2300.100 ZRXPCREATORKiIOSystem.ZRXP...,597205,Tylsen,Salzwedeler Dumme,Q.DGJ,Q.DGJ,UTC+1,-777,m³/s,"(timestamp,value,status,interpolation_type,rem...",...,597205,Day.Mean.DGJ,5854086,636623,Wehr,28.10,ZRXPVERSION2300.100 ZRXPCREATORKiIOSystem.ZRXP...,W.DGJ,W.DGJ,cm
124,ZRXPVERSION2300.100 ZRXPCREATORKiIOSystem.ZRXP...,597208,Salzwedel-Dumme,Salzwedeler Dumme,Q.DGJ,Q.DGJ,UTC+1,-777,m³/s,"(timestamp,value,status,interpolation_type,rem...",...,597208,Day.Mean.DGJ,5857978,644673,NaN,19.18,ZRXPVERSION2300.100 ZRXPCREATORKiIOSystem.ZRXP...,W.DGJ,W.DGJ,cm


Get all status

In [13]:
data = []
status = []
int_type = []

for m, df in zip(meta, raw_data):
    # get status
    for s in df.status.unique():
        if s not in status:
            status.append(s)
    
    # get interpolation types
    for t in df.interpolation_type.unique():
        if t not in int_type:
            int_type.append(t)
    
    # map status to quality flag (40: True, rest: NA)
    df['flag'] = df.status.map({40: True})

    # make the df
    out = df.iloc[:, :2].copy()
    out.columns = ['date', 'q' if m['CNAME'].startswith('Q') else 'w']
    out['flag'] = df['flag']
    data.append(out)

print(f"Status:              {status}")
print(f"Interpolation types: {int_type}")

Status:              [40, 552, -2147483393, 1064, 3112, -2147482369, 200, -2147482881, -2147483608]
Interpolation types: [603, 601, 604]


### Finally run

Now, the Q and W data can be extracted. The cool thing is, that all the id creation, data creation, merging and the mapping from our ids to the original ids and files is done by the context. This is helpful, as we less likely screw something up.

In [14]:
with Bundesland('Sachsen-Anhalt') as bl:
    # save the metadata
    bl.save_raw_metadata(qw_metadata, id_column, overwrite=True)

    # for reference, call the nuts-mapping as table
    nuts_map = bl.nuts_table
    print(nuts_map.head())

    # go for each    
    for m, df in tqdm(zip(meta, data), total=len(meta)):
        # get the provider id
        provider_id = str(m[id_column])
        bl.save_timeseries(df, provider_id,)

    # check if there were warnings (there are warnings)
    if len(warn) > 0:
        log_path = bl.save_warnings(warn)
        print(f"There were warnings during the processing. The log can be found at: {log_path}")


    nuts_id provider_id                              path
0  DEE10000      440003  ./DEE/DEE10000/DEE10000_data.csv
1  DEE10010      440004  ./DEE/DEE10010/DEE10010_data.csv
2  DEE10020      440008  ./DEE/DEE10020/DEE10020_data.csv
3  DEE10030      440010  ./DEE/DEE10030/DEE10030_data.csv
4  DEE10040      441201  ./DEE/DEE10040/DEE10040_data.csv


100%|██████████| 252/252 [00:17<00:00, 14.49it/s]
